In [1]:
pip install sastrawi


   ---------------------------------------- 209.7/209.7 kB 2.5 MB/s eta 0:00:00


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re 
import random 
import numpy as np

In [9]:
df = pd.read_csv('C:/Users/Reyhan Dwi/Documents/Reyhan/Bangkit Academy/Final Capstone/artikel-scrapping.csv')
df.rename(columns={df.columns[0]:'Kode'},inplace=True)
reco = df.copy()

In [11]:
reco['content'] = reco['content'] + ' ' + reco['title']

In [13]:
reco.head()

,Kode,title,content,date_created,author,link
0,0,Trend Kemasan Berkelanjutan: Botol Plastik Tan...,Industri pengemasan atau packaging saat ini me...,1 February 2024,admin,https://waste4change.com/blog/trend-kemasan-be...
1,1,Pemenang UN SDG Action Awards 2023 Bantu Kuran...,"Green Obsession, penerima penghargaan UN SDG A...",22 December 2023,Mita Defitri,https://waste4change.com/blog/pemenang-un-sdg-...
2,2,10 Cara Mengatasi Pencemaran Tanah dan Panduannya,Penting sekali untuk mengetahui cara mengatasi...,19 December 2023,Ganjar Nugraha,https://waste4change.com/blog/cara-mengatasi-p...
3,3,Apa itu Limbah Domestik? Berikut Contoh Jenis ...,Apa itu limbah domestik? Apa pengertian dari l...,11 December 2023,Ganjar Nugraha,https://waste4change.com/blog/apa-itu-limbah-d...
4,4,21 Contoh Limbah Keras Anorganik + Pengertian ...,"Sehari-hari, manusia selalu menghasilkan berba...",10 December 2023,Ganjar Nugraha,https://waste4change.com/blog/limbah-keras-ano...


In [16]:
sastrawi = StopWordRemoverFactory()
stopwords = sastrawi.get_stop_words()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [17]:
#Membersihkan Dataset
def clean_text(text):
  text = text.strip() #Menghapus spasi dan tab
  text = re.sub('[^a-z ]','',text) #Menghapus semua tanda baca dan numerik
  text = stemmer.stem(text) #Melakukan Stemming pada Text
  text = ' '.join(word for word in text.split() if word not in stopwords) #Menambahkan Spasi untuk setiap kata
  return text.strip()

In [18]:
df.content = df.content.apply(clean_text)

In [23]:
tf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1,3),min_df=0,use_idf=True)
tfidf_vector  = tf.fit_transform((df.content)).toarray()

user_input = 'bagaimana cara mengolah sampah plastik'
user_input = clean_text(user_input)
user_vector = tf.transform([user_input]).toarray()

In [21]:
def recommendations(input_user):
    user_input = clean_text(input_user)
    vector_user = tf.transform([user_input]).toarray()
    print(user_input)
    cos_sim = cosine_similarity(tfidf_vector,vector_user)
    df['cosim'] = cos_sim

    return df.sort_values('cosim',ascending=False).head()

In [24]:
recommendations('bagaimana cara mengelola sampah plastik')

bagaimana cara kelola sampah plastik


,Kode,title,content,date_created,author,link,cosim
50,50,Mengenal Lebih Dekat Sampah Plastik Fleksibel ...,enurut data ementerian ingkungan idup ehutanan...,26 August 2022,Mita Defitri,https://waste4change.com/blog/mengenal-lebih-d...,0.108769
18,18,Langkah China dalam Memerangi Sampah Plastik,hina sejak beberapa tahun lalu nobat jadi sala...,1 July 2023,Mita Defitri,https://waste4change.com/blog/langkah-china-da...,0.099410
46,46,Ketahui Manfaat Pengelolaan Sampah Rumah Tangga,ampah rumah tangga rupa salah satu masalah mun...,25 October 2022,Mita Defitri,https://waste4change.com/blog/ketahui-manfaat-...,0.095924
59,59,"Bahaya Sampah Plastik bagi Kesehatan, Temuan M...",ahaya sampah plastik makin hantu hidup manusia...,10 August 2022,Mita Defitri,https://waste4change.com/blog/bahaya-sampah-pl...,0.087116
11,11,Sampah Plastik di Korea Selatan: Cara Mengatas...,orea elatan keluar beberapa bijak tanggulang m...,9 August 2023,Mita Defitri,https://waste4change.com/blog/sampah-plastik-d...,0.086149
